In [32]:
# Import des librairies utiles 
import numpy as np
import pandas as pd
from scipy.optimize import least_squares
import Heston_pricer as hp  
import yfinance as yf
from datetime import datetime

In [33]:
#Étape 1 : On récupère les données de marché 

ticker = 'SPY' #Tciker du S&P 500 sur yfinance
spy = yf.Ticker(ticker)

# Prix spot
spot_price = spy.history(period='1d')['Close'].iloc[-1]

# Récupérer les options calls
expirations = spy.options
all_calls = []

for exp in expirations:
    chain = spy.option_chain(exp)
    calls = chain.calls.copy()
    calls['expirationDate'] = pd.to_datetime(exp)
    all_calls.append(calls)

df_calls = pd.concat(all_calls, ignore_index=True)

In [34]:
# Étape 1 bis : Retraiement et nettoyage des données

# Retirer les options illiquides
df_calls = df_calls[(df_calls['bid'] > 0) & (df_calls['ask'] > 0)]

# Calcul du prix mid (bid-ask moyen)
df_calls['Mid'] = (df_calls['bid'] + df_calls['ask']) / 2

# Calcul de la maturité en années
today = pd.Timestamp(datetime.utcnow().date())
df_calls['T'] = (df_calls['expirationDate'] - today).dt.days / 365.0

# Retirer les options déjà expirées ou trop proches (T < 7 jours)
df_calls = df_calls[df_calls['T'] > 7 / 365]

# Conserver uniquement les colonnes utiles pour calibration
df_heston = df_calls[['strike', 'Mid', 'T', 'expirationDate']]
df_heston = df_heston.rename(columns={'strike': 'Prix d\'exercice'})
df_heston['Spot'] = spot_price  # Ajout du spot constant

# Réorganiser
df_heston = df_heston[['Spot', 'Prix d\'exercice', 'T', 'Mid', 'expirationDate']]

# Affichage final
df_heston

/var/folders/q_/sr5xfkd12298qb_00f1_s20m0000gn/T/ipykernel_7331/1127404945.py:10: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  today = pd.Timestamp(datetime.utcnow().date())


,Spot,Prix d'exercice,T,Mid,expirationDate
361,623.619995,420.0,0.032877,204.240,2025-07-25
362,623.619995,450.0,0.032877,175.165,2025-07-25
363,623.619995,470.0,0.032877,154.325,2025-07-25
364,623.619995,480.0,0.032877,144.335,2025-07-25
365,623.619995,485.0,0.032877,139.410,2025-07-25
...,...,...,...,...,...
3321,623.619995,895.0,2.430137,4.485,2027-12-17
3322,623.619995,900.0,2.430137,4.195,2027-12-17
3323,623.619995,905.0,2.430137,3.925,2027-12-17
3324,623.619995,910.0,2.430137,3.680,2027-12-17


In [35]:
#Étape 2 : Tri des données pour le calibrage 
[#Ajout des colonnes utiles
df_heston['Moneyness'] = df_heston['Prix d\'exercice'] / df_heston['Spot']

#Filtrage brut
df_heston = df_heston[
    (df_heston['Mid'] > 0) &
    (df_heston['T'] >= 60 / 365) &  # >= 2 mois
    (df_heston['T'] <= 2) &         # <= 2 ans
    (df_heston['Moneyness'] >= 0.8) &
    (df_heston['Moneyness'] <= 1.2)
].copy()

#Catégorisation des maturités
def maturity_bucket(T):
    if T < 0.25:
        return 'short'
    elif T < 1.0:
        return 'medium'
    else:
        return 'long'

df_heston['maturity_bucket'] = df_heston['T'].apply(maturity_bucket)

#Catégorisation du moneyness
def moneyness_bucket(m):
    if m < 0.95:
        return 'ITM'
    elif m > 1.05:
        return 'OTM'
    else:
        return 'ATM'

df_heston['moneyness_bucket'] = df_heston['Moneyness'].apply(moneyness_bucket)

# Échantillonnage équilibré
df_heston = (
    df_heston
    .groupby(['maturity_bucket', 'moneyness_bucket'])
    .apply(lambda x: x.sample(n=min(len(x), 30), random_state=42))  # 30 max par groupe
    .reset_index(drop=True)
)

df_heston

/var/folders/q_/sr5xfkd12298qb_00f1_s20m0000gn/T/ipykernel_7331/2846594742.py:42: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(n=min(len(x), 30), random_state=42))  # 30 max par groupe


,Spot,Prix d'exercice,T,Mid,expirationDate,Moneyness,maturity_bucket,moneyness_bucket
0,623.619995,650.0,1.509589,52.035,2027-01-15,1.042301,long,ATM
1,623.619995,600.0,1.432877,82.170,2026-12-18,0.962124,long,ATM
2,623.619995,605.0,1.509589,80.510,2027-01-15,0.970142,long,ATM
3,623.619995,625.0,1.509589,67.140,2027-01-15,1.002213,long,ATM
4,623.619995,615.0,1.432877,72.055,2026-12-18,0.986177,long,ATM
...,...,...,...,...,...,...,...,...
265,623.619995,655.0,0.186301,4.125,2025-09-19,1.050319,short,OTM
266,623.619995,720.0,0.216438,0.095,2025-09-30,1.154549,short,OTM
267,623.619995,663.0,0.216438,3.085,2025-09-30,1.063147,short,OTM
268,623.619995,669.0,0.216438,2.155,2025-09-30,1.072769,short,OTM


In [36]:
# étape 3 : simuler le prix des options de marchés avec le modèle de Heston 

r=0.02 # risk free

# paramètres initiaux, non calibrés
sigma = 0.5
kappa = 1
theta= 0.05
volvol= 0.025
rho =-0.5


# Calcul du prix avec le modèle de Heston avec les paramètres non calibré
df_heston["Heston_price"] = df_heston.apply(
    lambda row: hp.call_priceHestonMid(
        row['Spot'],
        row["Prix d'exercice"],
        r,
        row["T"],
        sigma,
        kappa,
        theta,
        volvol,
        rho
    ),
    axis=1
)
df_heston

,Spot,Prix d'exercice,T,Mid,expirationDate,Moneyness,maturity_bucket,moneyness_bucket,Heston_price
0,623.619995,650.0,1.509589,52.035,2027-01-15,1.042301,long,ATM,47.503341
1,623.619995,600.0,1.432877,82.170,2026-12-18,0.962124,long,ATM,74.520691
2,623.619995,605.0,1.509589,80.510,2027-01-15,0.970142,long,ATM,73.346795
3,623.619995,625.0,1.509589,67.140,2027-01-15,1.002213,long,ATM,61.092039
4,623.619995,615.0,1.432877,72.055,2026-12-18,0.986177,long,ATM,64.971954
...,...,...,...,...,...,...,...,...,...
265,623.619995,655.0,0.186301,4.125,2025-09-19,1.050319,short,OTM,5.124477
266,623.619995,720.0,0.216438,0.095,2025-09-30,1.154549,short,OTM,-0.012697
267,623.619995,663.0,0.216438,3.085,2025-09-30,1.063147,short,OTM,4.347465
268,623.619995,669.0,0.216438,2.155,2025-09-30,1.072769,short,OTM,3.303397


In [37]:
# Étape 4 : focntion residulas qui correspond à la fonction à minimiser (erreur)

def residuals(params, df, r=0.02):
    kappa, theta, sigma, rho, volvol = params
    res = []

    for i, row in df.iterrows():
        try:
            S = row['Spot']
            K = row["Prix d'exercice"]
            T = row['T']
            market_price = row['Mid']

            # --- Calcul du prix modèle (Heston) ---
            model_price = hp.call_priceHestonMid(S, K, r, T, sigma, kappa, theta, volvol, rho)
        
            res.append((model_price - market_price)) 
        except Exception as e:
            print(f"Erreur ligne {i}: {e}")
            res.append(1e6)
    return np.array(res)

In [38]:
#Étape 5 : Minimisation de l'erreur en utlisant la méthode de Levenberg-Maquardt
from scipy.optimize import least_squares

#Paramètres initiaux 
init_params = [1.0, 0.05, 0.5, -0.5, 0.025]  # [kappa, theta, sigma, rho, volvol]

# --- Lancement de la calibration ---
print("Lancement de la calibration ")

result = least_squares(
    residuals,
    init_params,
    args=(df_heston,),
    method='lm',          # méthode Levenberg-Marquardt
    verbose=2
)

# --- Résultats ---
kappa, theta, sigma, rho, volvol = result.x
rmse = np.sqrt(np.mean(result.fun**2))

print("\n Calibration terminée.")
print(f"Paramètres calibrés :\n kappa={kappa:.4f}, theta={theta:.4f}, "
      f"sigma={sigma:.4f}, rho={rho:.4f}, volvol={volvol:.4f}")
print(f"RMSE : {rmse:.4f}")


Lancement de la calibration 
`ftol` termination condition is satisfied.
Function evaluations 58, initial cost 3.4358e+03, final cost 1.1857e+03, first-order optimality 4.26e-01.

 Calibration terminée.
Paramètres calibrés :
 kappa=0.7253, theta=0.1084, sigma=0.7848, rho=-0.7784, volvol=0.0299
RMSE : 2.9636
